In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.core import *
from local.data.all import *
from local.notebook.showdoc import show_doc

In [ ]:
#export
pd.set_option('mode.chained_assignment','raise')

In [ ]:
#default_exp tabular.core

# Tabular core

> Basic function to preprocess tabular data before assembling it in a `DataBunch`.

## TabularProc -

We use this class to preprocess tabular data. `cat_names` should contain the names of the categorical variables in your dataframe, `cont_names` the names of the continuous variables. If you don't need any state, you can initiliaze a `TabularProc` with a `func` to be applied on the dataframes. Otherwise you should subclass and implement `setup` and `__call__`.

In [ ]:
#export
class Tabular(CollBase):
    def __init__(self, df, cat_names=None, cont_names=None, y_names=None, is_y_cat=True, splits=None):
        super().__init__(df)
        self.splits = L(ifnone(splits,slice(None)))
        self.cat_names,self.cont_names,self.y_names = L(cat_names),L(cont_names),y_names
        self.cat_y  = None if not is_y_cat else y_names
        self.cont_y = None if     is_y_cat else y_names

    def __setitem__(self,k,v): super().__setitem__(list(k) if isinstance(k,L) else k, v)
    def transform(self, cols, f): self[cols] = self[cols].transform(f)

    @property
    def loc(self): return self.items.loc
    @property
    def iloc(self): return self.items.iloc

    @property
    def all_cont_names(self): return self.cont_names + self.cont_y
    @property
    def all_cat_names (self): return self.cat_names  + self.cat_y
    @property
    def all_col_names (self): return self.all_cont_names + self.all_cat_names

In [ ]:
#export
def _add_prop(cls, nm):
    prop = property(lambda o: o.items[list(getattr(o,nm+'_names'))])
    setattr(cls, nm+'s', prop)
    def _f(o,v): o.items[list(getattr(o,nm+'_names'))] = v
    setattr(cls, nm+'s', prop.setter(_f))

_add_prop(Tabular, 'cat')
_add_prop(Tabular, 'all_cat')
_add_prop(Tabular, 'cont')
_add_prop(Tabular, 'all_cont')
_add_prop(Tabular, 'all_col')

In [ ]:
#export
class TabularProc(InplaceTransform):
    "Base class to write a tabular processor for dataframes"
    def process(self, *args,**kwargs): return self(*args,**kwargs)

In [ ]:
t = (0,1,2)
class Categorify(TabularProc, CollBase):
    "Transform the categorical variables to that type."
    order = 1
    def setup(self, to):
        to.classes = self.items = {n:CategoryMap(to.loc[to.splits[0],n], add_na=True)
                                   for n in to.all_cat_names}

    def _apply_cats(self, c): return c.cat.codes+1 if is_categorical_dtype(c) else c.map(self[c.name].o2i)
    def encodes(self, to): to.transform(to.all_cat_names, self._apply_cats)
    def decodes(self, to):
        cats = [self[c][v] for v,c in zip(to.items[0], to.cat_names)]
        to.items = (cats, to.items[1])
        return to

In [ ]:
t[2:],u[2:]

In [ ]:
#export
class Categorify(TabularProc, CollBase):
    "Transform the categorical variables to that type."
    order = 1
    def setup(self, to):
        to.classes = self.items = {n:CategoryMap(to.loc[to.splits[0],n], add_na=True)
                                   for n in to.all_cat_names}

    def _apply_cats(self, c): return c.cat.codes+1 if is_categorical_dtype(c) else c.map(self[c.name].o2i)
    def encodes(self, to): to.transform(to.all_cat_names, self._apply_cats)
    def decodes(self, to):
        cats = [[self[c][v] for v,c in zip(t, to.cat_names)] for t in to.items[0]]
        cat_y = to.items[2] if to.cat_y is None else [self[to.cat_y][v] for v in to.items[2]]
        to.items = (cats,to.items[1],cat_y)
        return to

In [ ]:
show_doc(Categorify, title_level=3)

In [ ]:
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,0,2]})
to = Tabular(df, 'a')

cat.setup(to)
test_eq(cat['a'], ['#na#',0,1,2])
cat(to)
test_eq(df['a'], [1,2,3,1,3])
df1 = pd.DataFrame({'a':[1,0,3,-1,2]})
to1 = Tabular(df1, 'a')
cat(to1)
#Values that weren't in the training df are sent to 0 (na)
test_eq(df1['a'], [2,1,0,0,3])

In [ ]:
#test with splits
cat = Categorify()
df = pd.DataFrame({'a':[0,1,2,3,2]})
to = Tabular(df, 'a', splits=[range(3)])
cat.setup(to)
test_eq(cat['a'], ['#na#',0,1,2])
cat(to)
test_eq(df['a'], [1,2,3,0,3])

In [ ]:
#test NaN
class Normalize(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setup(self, to):
        df = to.loc[to.splits[0], to.cont_names]
        self.means,self.stds = df.mean(),df.std(ddof=0)

    def encodes(self, to): to.conts = (to.conts-self.means) / (self.stds+1e-7)
    def decodes(self, to):
        conts = [(v*self.stds[c] + self.means[c]).item() for v,c in zip(to.items[1], to.cont_names)]
        to.items = (to.items[0], conts)
        return to

In [ ]:
df = pd.DataFrame({'a':pd.Categorical(['M','H','L','M'], categories=['H','M','L'], ordered=True)})
to = Tabular(df, 'a')
cat = Categorify()
cat.setup(to)
test_eq(cat['a'], ['#na#','H','M','L'])
cat(to)
test_eq(df.a, [2,1,3,2])

In [ ]:
#export
class Normalize(TabularProc):
    "Normalize the continuous variables."
    order = 2
    def setup(self, to):
        df = to.loc[to.splits[0], to.cont_names]
        self.means,self.stds = df.mean(),df.std(ddof=0)

    def encodes(self, to): to.conts = (to.conts-self.means) / (self.stds+1e-7)
    def decodes(self, to):
        conts = [[self.stds[c] * v.item() + self.means[c] for c,v in zip(to.cont_names, t)] for t in to.items[1]]
        to.items = (to.items[0], conts, to.items[2])
        return to

In [ ]:
show_doc(Normalize, title_level=3)

In [ ]:
norm = Normalize()
class FillStrategy:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return c.median()
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().idxmax()

In [ ]:
norm = Normalize()
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr(self, 'fill_strategy,add_col,fill_vals')

    def setup(self, to):
        df = to.loc[to.splits[0], to.cont_names]
        self.na_dict = {n:self.fill_strategy(df[n], self.fill_vals[n])
                        for n in pd.isnull(to.conts).any().keys()}

    def encodes(self, to):
        missing = pd.isnull(to.conts)
        for n in missing.any().keys():
            assert n in self.na_dict, f"nan values in `{n}` but not in setup training set"
            to[n].fillna(self.na_dict[n], inplace=True)
            if self.add_col:
                to[n+'_na'] = missing[n]
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')

In [ ]:
#export
class FillStrategy:
    "Namespace containing the various filling strategies."
    def median  (c,fill): return c.median()
    def constant(c,fill): return fill
    def mode    (c,fill): return c.dropna().value_counts().idxmax()

In [ ]:
#export
class FillMissing(TabularProc):
    "Fill the missing values in continuous columns."
    def __init__(self, fill_strategy=FillStrategy.median, add_col=True, fill_vals=None):
        if fill_vals is None: fill_vals = defaultdict(int)
        store_attr(self, 'fill_strategy,add_col,fill_vals')

    def setup(self, to):
        df = to.loc[to.splits[0], to.cont_names]
        self.na_dict = {n:self.fill_strategy(df[n], self.fill_vals[n])
                        for n in pd.isnull(to.conts).any().keys()}

    def encodes(self, to):
        missing = pd.isnull(to.conts)
        for n in missing.any().keys():
            assert n in self.na_dict, f"nan values in `{n}` but not in setup training set"
            to[n].fillna(self.na_dict[n], inplace=True)
            if self.add_col:
                to[n+'_na'] = missing[n]
                if n+'_na' not in to.cat_names: to.cat_names.append(n+'_na')

In [ ]:
show_doc(FillMissing, title_level=3)

In [ ]:
fill1,fill2,fill3 = (FillMissing(fill_strategy=s) 
                     for s in [FillStrategy.median, FillStrategy.constant, FillStrategy.mode])
df = pd.DataFrame({'a':[0,1,np.nan,1,2,3,4]})
df1 = df.copy(); df2 = df.copy()
to,to1,to2 = Tabular(df, cont_names='a'),Tabular(df1, cont_names='a'),Tabular(df2, cont_names='a')
fill1.setup(to); fill2.setup(to1); fill3.setup(to2)
test_eq(fill1.na_dict, {'a': 1.5})
test_eq(fill2.na_dict, {'a': 0})
test_eq(fill3.na_dict, {'a': 1.0})

fill1(to); fill2(to1); fill3(to2)
for t in [to, to1, to2]: test_eq(t.cat_names, ['a_na'])

for df_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([0, 1, v, 1, 2, 3, 4]))
    test_eq(df_['a_na'].values, np.array([0, 0, 1, 0, 0, 0, 0]))
    
dfa = pd.DataFrame({'a':[np.nan,0,np.nan]})
dfa1 = dfa.copy(); dfa2 = dfa.copy()
to,to1,to2 = Tabular(dfa, cont_names='a'),Tabular(dfa1, cont_names='a'),Tabular(dfa2, cont_names='a')
fill1(to); fill2(to1); fill3(to2)
for df_,v in zip([to, to1, to2], [1.5, 0., 1.]):
    test_eq(df_['a'].values, np.array([v, 0, v]))
    test_eq(df_['a_na'].values, np.array([1, 0, 1]))

## Tabular Pipelines -

In [ ]:
procs = [Normalize(), Categorify(), FillMissing(), noop]
proc = Pipeline(procs)

#Test reordering and partialize
test_eq(L(proc.fs).mapped(type), [FillMissing, Transform, Categorify, Normalize])

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4]})
to = Tabular(df, 'a', 'b')

#Test setup and apply on df_trn
proc.setup(to)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to['a'], [1,2,3,2,2,3,1])
test_eq(to['b_na'], [1,1,2,1,1,1,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to['b'].values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True]})

In [ ]:
#Test apply on df_val
@delegates(Tabular)
def process_df(df, procs, inplace=True, **kwargs):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessor` associated"
    to = Tabular(df if inplace else df.copy(), **kwargs)
    proc = Pipeline(procs)
    proc.setup(to)
    return to,proc

In [ ]:
#Test apply on y_names
procs = [Normalize(), Categorify(), FillMissing(), noop]
proc = Pipeline(procs)

df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to = Tabular(df, 'a', 'b', y_names='c')
proc.setup(to)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to['a'], [1,2,3,2,2,3,1])
test_eq(to['b_na'], [1,1,2,1,1,1,1])
test_eq(to['c'], [2,1,2,1,1,2,1])
x = np.array([0,1,1.5,1,2,3,4])
m,s = x.mean(),x.std()
test_close(to['b'].values, (x-m)/s)
test_eq(proc.classes, {'a': ['#na#',0,1,2], 'b_na': ['#na#',False,True], 'c': ['#na#','a','b']})

In [ ]:
#export
@delegates(Tabular)
def process_df(df, procs, inplace=True, **kwargs):
    "Process `df` with `procs` and returns the processed dataframe and the `TabularProcessor` associated"
    to = Tabular(df if inplace else df.copy(), **kwargs)
    proc = Pipeline(procs)
    proc.setup(to)
    return to,proc

In [ ]:
procs = [Normalize(), Categorify(), FillMissing(), noop]
df = pd.DataFrame({'a':[0,1,2,1,1,2,0], 'b':[0,1,np.nan,1,2,3,4], 'c': ['b','a','b','a','a','b','a']})
to,proc = process_df(df, procs, cat_names='a', cont_names='b', y_names='c', inplace=False)
test_eq(to.cat_names, ['a', 'b_na'])
test_eq(to['a'], [1,2,3,2,2,3,1])
test_eq(df.a.dtype,int)
test_eq(to['b_na'], [1,1,2,1,1,1,1])
test_eq(to['c'], [2,1,2,1,1,2,1])

In [ ]:
# inplace
class TabularLine(pd.Series):
    "A line of a dataframe that knows how to show itself"
    def show(self, ctx=None, **kwargs): return self if ctx is None else ctx.append(self)

Pass the same `splits` as you will use for splitting the data, so that the setup is only done on the training set. `cat_names` are the names of the categorical variables, `cont_names` the continous ones, `y_names` are the names of the dependent variables that are categories. If `inplace=True`, processing is applied inplace, otherwis it creates a copy of `df`.
class TensorTabular(tuple):
    def get_ctxs(self, max_n=10, **kwargs):
        n_samples = min(self[0].shape[0], max_n)
        df = pd.DataFrame(index = range(n_samples))
        return [df.iloc[i] for i in range(n_samples)]

    def display(self, ctxs): display_df(pd.DataFrame(ctxs))

In [ ]:
#export
class ReadTabLine(ItemTransform):
    def __init__(self, proc): self.proc = proc

    def encodes(self, row):
        cats,conts = (o.mapped(row.__getitem__) for o in (self.proc.cat_names,self.proc.cont_names))
        return TensorTabular((tensor(cats).long(),tensor(conts).float()))

    def decodes(self, o) -> TabularLine:
        to = Tabular(o, self.proc.cat_names, self.proc.cont_names, self.proc.y_names)
        to = self.proc.decode(to)
        return pd.Series({c: v for v,c in zip(to.items[0]+to.items[1], self.proc.cat_names+self.proc.cont_names)})

In [ ]:
#export
class ReadTabTarget(ItemTransform):
    def __init__(self, proc): self.proc = proc
    def encodes(self, row): return row[self.proc.y_names].astype(np.int64)
    def decodes(self, o) -> Category: return self.proc.classes[self.proc.y_names][o]

In [ ]:
#export
class ReadTabLine(ItemTransform):
    def __init__(self, proc): self.proc = proc

    def encodes(self, row):
        cats,conts = (o.mapped(row.__getitem__) for o in (self.proc.cat_names,self.proc.cont_names))
        return TensorTabular((tensor(cats).long(),tensor(conts).float()))

    def decodes(self, o) -> TabularLine:
        to = Tabular(([o[0]], [o[1]], [0]), self.proc.cat_names, self.proc.cont_names, None)
        to = self.proc.decode(to)
        return pd.Series({c: v for v,c in zip(to.items[0][0]+to.items[1][0], self.proc.cat_names+self.proc.cont_names)})

In [ ]:
#export
class ReadTabTarget(ItemTransform):
    def __init__(self, proc): self.proc = proc
    def encodes(self, row): return row[self.proc.y_names].astype(np.int64)
    def decodes(self, o) -> Category: return self.proc.classes[self.proc.y_names][o]

In [ ]:
tds = TfmdDS(to.items, tfms=[[ReadTabLine(proc)], ReadTabTarget(proc)])
enc = tds[1]
test_eq(enc[0][0], tensor([2,1]))
test_close(enc[0][1], tensor([-0.628828]))
test_eq(enc[1], 1)

dec = tds.decode(enc)
assert isinstance(dec[0], TabularLine)
test_close(dec[0], pd.Series({'a': 1, 'b_na': False, 'b': 1}))
test_eq(dec[1], 'a')

test_stdout(lambda: print(tds.show_at(1)), """a               1
b_na        False
b               1
category        a
dtype: object""")

## Integration example

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
df_trn,df_tst = df.iloc[:10000].copy(),df.iloc[10000:].copy()
df_trn.head()

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [Categorify(), FillMissing(), Normalize()]

In [ ]:
splits = RandomSplitter()(range_of(df_trn))
to,proc = process_df(df_trn, procs, splits=splits, cat_names=cat_names, cont_names=cont_names, y_names="salary")

In [ ]:
dsrc = DataSource(to.items, filts=splits, tfms=[[ReadTabLine(proc)], [ReadTabTarget(proc)]])
class ReadTabBatch(ItemTransform):
    def __init__(self, proc): self.proc = proc

    def encodes(self, df):
        cats,conts,targ = (df[o] for o in (self.proc.cat_names,self.proc.cont_names,self.proc.y_names))
        return (TensorTabular((tensor(cats.values).long(),tensor(conts.values).float())), tensor(targ.values).long())

    def decodes(self, o):
        (cats,conts),targ = o
        res = []
        for cat,cont,t in zip(cats,conts,targ):
            to = Tabular((cat,cont), self.proc.cat_names, self.proc.cont_names, self.proc.y_names)
            to = self.proc.decode(to)
            res.append(pd.Series({c: v for v,c in zip(to.items[0]+to.items[1], self.proc.cat_names+self.proc.cont_names)}))
        return pd.DataFrame(res)

In [ ]:
dbch = dsrc.databunch(bs=64, num_workers=0)
%time _ = L(dbch.valid_dl)

In [ ]:
class TabularDF(pd.DataFrame):
    def show(self, max_n=10, **kwargs): display_df(self.iloc[:max_n])

In [ ]:
#export
class ReadTabBatch(ItemTransform):
    def __init__(self, proc): self.proc = proc

    def encodes(self, df):
        cats,conts,targ = (df[o] for o in (self.proc.cat_names,self.proc.cont_names,self.proc.y_names))
        return (TensorTabular((tensor(cats.values).long(),tensor(conts.values).float())), tensor(targ.values).long())

    def decodes(self, o)->TabularDF:
        (cats,conts),targs = o
        res = []
        to = Tabular((cats,conts,targs), self.proc.cat_names, self.proc.cont_names, self.proc.y_names, is_y_cat=self.proc.cat_y is not None)
        to = self.proc.decode(to)
        return pd.DataFrame({**{c: [t[i] for t in to.items[0]] for i,c in enumerate(self.proc.cat_names)},
                             **{c: [t[i] for t in to.items[1]] for i,c in enumerate(self.proc.cont_names)},
                             self.proc.y_names: [t for t in to.items[2]]})

In [ ]:
class TabDataLoader(TfmdDL):
    do_item = noops
    def create_batch(self, b): return self.dataset.items.iloc[b]

In [ ]:
dl = TabDataLoader(dsrc.valid, bs=64, after_batch = ReadTabBatch(proc))

In [ ]:
b = dl.one_batch()

In [ ]:
dl.show_batch()

In [ ]:
to_tst = Tabular(df_tst, cat_names, cont_names, y_names="salary")
proc(to_tst)
to_tst.all_cols.head()

## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)